# Compsci 690V Homework 5

We pick the mini challenge 1 of VAST Challenge 2008 for this homework. 

**Description:** The Paraiso movement is controversial and is having considerable social impact in a specific area of the world. We have extracted a segment of the Paraiso (the movement) Wikipedia edits page. Please note this is not the Paraiso Manifesto Wiki page which is part of the background materials, but a related different page. Please use visual analytics to describe the social relationships of the editors (those that have edited/modified the Wikipedia page) as they are reflected in these files.



To obtain a dataset similar to the one given in the mini challenge, we scraped the edits of the wikipedia page **"2016–17 Kashmir unrest"** (link: https://en.wikipedia.org/w/index.php?title=2016%E2%80%9317_Kashmir_unrest&limit=500&action=history). We chose this dataset since it is a recent revolutionary movement simirarly to the one described in the challenge. We scraped the text using the script in the file **getEdits.py**.

We parsed the text of each commit into a dataframes with the columns:
* **timestamp** - the timestamp of the edit
* **user** - the user or ip of the editor
* **minorEdit** - True or False - if the edit is minor
* **pageLength** - Length of the page after the edit (in Bytes)
* **editDiff** - bytes changed by the edit
* **comment** - comment or description of the edit
* **tags**
* **entireEdit** - entire raw edit text

In [73]:
import numpy as np
import pandas as pd
import re
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, Select,LabelSet, HoverTool
import nltk
import re
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

output_notebook()


file = open('data/wiki sample data.txt','r', errors='ignore')

lines = file.readlines()
#print (lines)

df = pd.DataFrame(columns=['timestamp','user','minorEdit','pageLength','editDiff','comment','tags','entireEdit'])
i=0
for line in lines:
        entireEdit = line
        #split via brackets
        token = re.split('\(|\)',line)
        timestamp = token[2].split('?')[0]
        timestamp = pd.to_datetime(timestamp)
        #split by ?
        user = token[2].split('?')[1:]
        user = ''.join(user)
        user = user[1:-1]
        m = token[4]
        m= m.split('.')[0]
        m= True if (m=='? m ') else False
        byte = token[5].split(' ')[0]
        byte = int(byte.replace(',',''))
        change = token[7]
        change = int(change.replace(',',''))
        comment = re.split('\. \. ',line)[-1]
        tagComment = comment.split('(undo)')
        comment = tagComment[0][1:-2]
        #undo is not at the end
        if (len(tagComment)>1):
            tag = tagComment[1][2:-2]
            tag = tag.split(':')
            if (len(tag)>1):
                tag = tag[1][1:]
                tag = tag.split(',')
            if (tag[0]==''):
                tag =[]
        else:
            tag=''

        df.loc[i]=[timestamp,user,m,byte,change,comment,tag,entireEdit]
        i+=1

print (df.shape)

Loading BokehJS ...

(1338, 8)


Next we look at the users who have made the most edits:

In [3]:
contributions = df['user'].value_counts()
print(contributions.head())

DinoBambinoNFS     437
Kautilya3          101
Support2016         56
117.214.245.178     29
Thnidu              26
Name: user, dtype: int64


Some of the edits have also been done by wikipedia bots. These bots usally have **'Bot'** at the end of their names:

In [4]:
for name in contributions.index.tolist():
    if('Bot' in name):
        print(name, contributions[name])

Bender the Bot 2
ClueBot NG 2
MusikBot 2
KolbertBot 1


We look at the edits that we just reverts of previous commits:

In [36]:
revertEdits = []
for row in df.iterrows():
    if('Revert' in row[1]['comment'] or 'revert' in row[1]['comment']):
        revertEdits.append(row)

print('Number of edits that are reverts: ',len(revertEdits))



Number of edits that are reverts:  83


In [51]:
userWiseComments = pd.DataFrame(columns=['user','allComments'])
users = list(set(df['user']))
i = 0
for user in users:
    userRows = df.loc[df['user'] == user]
    allComments = ''
    for row in userRows.iterrows():
        allComments += row[1]['comment']
    userWiseComments.loc[i] = [user,allComments]
    i += 1

In [83]:



stemmer = SnowballStemmer("english")

#take comments from dataframe
commentsDf = userWiseComments['allComments']

#token given text and find their stems
def tokenizeAndStem(text):
    CommentsTokens=[]
    #for userComments in commentsDf:
    CommentsTokens = (nltk.word_tokenize(text))
    #filter out punctuations and numeric tokens
    filtered_tokens = []
    for token in CommentsTokens:
        if re.search('[a-zA-Z0-9]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

#create stem dictionary for all the commments
totalvocab_stemmed = []
for i in commentsDf:
    stem = tokenizeAndStem(i)
    totalvocab_stemmed.extend(stem)

print (len(totalvocab_stemmed))

#making Term Frequency-Inverse document Frequency matrix model
tfidf_vectorizer = TfidfVectorizer(max_df=0.8,
                                 min_df=0, stop_words='english',
                                 use_idf=False,tokenizer=tokenizeAndStem, ngram_range=(1,4))  #creating features by taking 3 words at a time

tfidf_matrix = tfidf_vectorizer.fit_transform(commentsDf)
print (tfidf_matrix.shape)
terms = tfidf_vectorizer.get_feature_names()
#terms are list of features in the matrix
#print (terms)



7149
(289, 12447)


In [96]:
from sklearn.cluster import KMeans,DBSCAN

num_clusters = 3
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

db = DBSCAN(eps=0.8,min_samples=2)
db.fit(tfidf_matrix)
clustersDB = db.labels_.tolist()
clusters = km.labels_.tolist()
#clusters is a list corresponding to 1338 rows in df
#print (clusters)
#add clusters to dataframe
#userWiseComments['clusters']  =clustersDB
#print (df.head())
#show the distribution of clusters
#print (df['clusters'].value_counts())

order_centroids = km.cluster_centers_.argsort()[:,::-1]
print (order_centroids)

for centres in order_centroids:
    topFeatures=[]
    for words in centres[:15]:
        topFeatures.append(terms[words])
    print (topFeatures)

[[ 1843  2335  9590 ...,  8269  8268     0]
 [11331  4581 11722 ..., 10861 10862  5798]
 [  448  5265  7974 ...,  8242  8241     0]]
['background', 'casualti', 'septemb', 'background background', 'background death', 'death', 'septemb background', 'septemb novemb', 'novemb', 'casualti accur', 'casualti accur figur', 'line', 'background reword line', 'background reword', 'figur']
['unrest', 'fix', 'use', 'talk', 'ad', 'edit', 'link', 'revis', 'kashmir', 'revert', 'reaction', 'typo', 'pellet', 'pellet gun', 'gun']
['2017', 'india', 'protest', 'articl', "n't", 'unrest', 'separatist', 'imag', 'reaction', 'pellet', 'mention', 'notabl', 'sourc', 'fund', 'chang']


In [97]:
from sklearn.manifold import MDS
#dimensionality reduction for the TF-IDF matrix (or df equally)
MDS()

# convert two components as we're plotting points in a two-dimensional plane

# we will also specify `random_state` so the plot is reproducible.
mds = MDS(n_components=2, random_state=1)

#toarray() converts sparse array to dense numpy array
pos = mds.fit_transform(tfidf_matrix.toarray())  # shape (n_components, n_samples)

#store the dimensions in xs, ys
xs, ys = pos[:, 0], pos[:, 1]


In [98]:


def get_colors(clusters):
    colors=[]
    for i in clusters:
        if i==-1:
            colors.append('violet')
        elif i==0:
            colors.append('red')
        elif i==1:
            colors.append('blue')
        elif i==2:
            colors.append('green')
        elif i==3:
            colors.append('brown')
        elif i==4:
            colors.append('yellow')
        elif i==5:
            colors.append('pink')
        elif i==6:
            colors.append('orange')
        else:
            colors.append('black')
    return colors

colors = get_colors(clustersDB)




# a=figure(x_range =list(set(list(df['user']))),plot_height=500, plot_width=900, title='clustering groups based on edits')
# data = ColumnDataSource(data=dict(x=df['user'],y=df['timestamp'],colors = colors,label=df['tags']))
# a.square('x','y',fill_color='colors',line_color='colors',source=data,size=10)
# #users are label
# labels = LabelSet(x='x', y='y', text='label', level='glyph',
#                   x_offset=5, y_offset=5, source=data, render_mode='canvas')
# #a.add_layout(labels)
# #show(a)

print(data.column_names)
hover = HoverTool(tooltips=[
    ("user",'@user'),
    ("comment","@comment")
])
b=figure( plot_height=500, plot_width=900, title='clustering groups based on edits', tools = [hover])
data1 = ColumnDataSource(data=dict(x=xs,y=ys,colors = colors, user = list(userWiseComments['user']), 
                                   comment = userWiseComments['allComments']))

b.square('x','y',fill_color='colors',line_color='colors',source=data1,size=10)
#user are labels
labels = LabelSet(x='x', y='y', text='label', level='glyph',
                  x_offset=5, y_offset=5, source=data1,render_mode='canvas')
#b.add_layout(labels)
show(b)

['x', 'y', 'colors', 'label']
